In [65]:
!pip install -r requirements.txt

In [66]:
import pandas as pd
import numpy as np
import http.client
import requests
import pandas as pd
import json
import pprint
import seaborn as sns
import matplotlib.pyplot as plt

In [67]:
diet_req = pd.read_csv("dietary_req.csv").drop(['Source'],axis=1)
#Dividing by 3 to reflect meal
diet_req = diet_req.set_index('Nutrition')
dietreq_meal=diet_req.div(3)
dietreq_meal.iloc[len(dietreq_meal)-1,:] = dietreq_meal.iloc[len(dietreq_meal)-1,:] * -1

In [68]:
#Deliverable A: Creating Population Diet Requirement Function
groups = ['M 19-30','F 19-30']
def create_pop(group,activity):
    pop_req = dietreq_meal[group]
    if activity:
        pop_req.loc[0,group]= pop_req.iloc[0,1:3] + 300
    return pop_req
group = create_pop(groups, True).drop(0,axis=0)
group

/tmp/ipykernel_24/1045228780.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  pop_req.loc[0,group]= pop_req.iloc[0,1:3] + 300


,M 19-30,F 19-30
Nutrition,,
Energy,800.000000,666.666667
Protein,18.666667,15.333333
"Fiber, total dietary",11.200000,9.333333
"Folate, DFE",133.333333,133.333333
"Calcium, Ca",333.333333,333.333333
"Carbohydrate, by difference",43.333333,43.333333
"Iron, Fe",2.666667,6.000000
"Magnesium, Mg",133.333333,103.333333
Niacin,5.333333,4.666667


In [98]:
price_nutrition = pd.read_csv('PriceData.csv')
price_nutrition=price_nutrition.set_index("Ingredients").drop(['Meal Name','Size','ID','fdcId','Brand'],axis=1)
price_nutrition[0:4]

,Energy,Protein,"Fiber, total dietary","Folate, DFE","Calcium, Ca","Carbohydrate, by difference","Iron, Fe","Magnesium, Mg",Niacin,"Phosphorus, P",...,Thiamin,"Vitamin A, RAE",Vitamin B-12,Vitamin B-6,"Vitamin C, total ascorbic acid",Vitamin E (alpha-tocopherol),Vitamin K (phylloquinone),"Zinc, Zn","Sodium, Na",Price
Ingredients,,,,,,,,,,,,,,,,,,,,,
Asparagus,18.0,2.35,2.4,52.0,24.0,3.53,2.12,14.0,0.98,52.0,...,0.143,38,0.0,0.091,5.9,1.13,41.6,0.54,2,1.88
Avacado Chunks,160.0,2.00,6.7,81.0,12.0,8.53,0.55,29.0,1.74,52.0,...,0.067,7,0.0,0.257,10.0,2.07,21.0,0.64,7,2.50
Flavored Brown Rice,106.0,2.76,1.2,10.0,7.0,19.30,0.39,37.0,1.68,89.0,...,0.071,16,0.0,0.106,0.0,0.35,2.4,0.99,293,6.00
Carrots,38.0,1.28,2.6,19.0,38.0,8.97,0.26,12.0,0.98,35.0,...,0.066,835,0.0,0.138,6.4,0.66,13.2,0.24,71,1.00


In [99]:
from  scipy.optimize import linprog as lp
def solve_subsistence_problem(FoodNutrients,nutrients, prices):
    A = price_nutrition[nutrients].apply(pd.to_numeric).T
    b = dietreq_meal.iloc[0,:]
    result = lp(prices, A, -b, method='interior-point')
    result.A = A
    result.b = b
    
    if result.success:
        result.diet = pd.Series(result.x,index=p.index)
    else: # No feasible solution?
        result.diet = pd.Series(result.x,index=p.index)*np.nan  
    return result

In [119]:
Prices = price_nutrition[np.append(group.index,'Price')]['Price'].to_list()
Prices = [float(s) for s in Prices] 
result = solve_subsistence_problem(price_nutrition,group.index, Prices)

ValueError: Invalid input for linprog: A_ub must have exactly two dimensions, and the number of columns in A_ub must be equal to the size of c